In [72]:
import json
import numpy as np
from nltk.util import everygrams
from nltk.util import ngrams

In [99]:
text = ['a', 'b', 'c', 'd', 'e', 'b', 'c']


In [100]:
list(ngrams(text, 3))

[('a', 'b', 'c'),
 ('b', 'c', 'd'),
 ('c', 'd', 'e'),
 ('d', 'e', 'b'),
 ('e', 'b', 'c')]

In [79]:
list(everygrams(text, max_len=3))

[('a',),
 ('b',),
 ('c',),
 ('d',),
 ('e',),
 ('b',),
 ('c',),
 ('a', 'b'),
 ('b', 'c'),
 ('c', 'd'),
 ('d', 'e'),
 ('e', 'b'),
 ('b', 'c'),
 ('a', 'b', 'c'),
 ('b', 'c', 'd'),
 ('c', 'd', 'e'),
 ('d', 'e', 'b'),
 ('e', 'b', 'c')]

In [9]:
def read_lm_data(file):
    res = ['</s>']
    with open(file) as fin:
        for i, line in enumerate(fin):
            if i % 100000 == 0:
                print(f'procesed {i} lines')
            for tok in line.strip().split():
                res.append(tok)
                
            res.append('</s>')
    
    return res

In [11]:
train_f = '/projects/junxianh/knnlmXS/examples/language_model/wikitext-103/wiki.train.tokens'

In [12]:
train_d = read_lm_data(train_f)

procesed 0 lines
procesed 100000 lines
procesed 200000 lines
procesed 300000 lines
procesed 400000 lines
procesed 500000 lines
procesed 600000 lines
procesed 700000 lines
procesed 800000 lines
procesed 900000 lines
procesed 1000000 lines
procesed 1100000 lines
procesed 1200000 lines
procesed 1300000 lines
procesed 1400000 lines
procesed 1500000 lines
procesed 1600000 lines
procesed 1700000 lines
procesed 1800000 lines


In [13]:
len(train_d)

103227022

In [14]:
from nltk.lm import Vocabulary

In [16]:
vocab = Vocabulary(train_d)

In [45]:
vocab['a']

1677048

In [19]:
vocab.lookup("jxxxxx")

'<UNK>'

In [36]:
ngram = 3
train = everygrams(train_d, max_len=ngram)

In [37]:
from nltk.lm import KneserNeyInterpolated

In [46]:
lm = KneserNeyInterpolated(order=ngram, vocabulary=vocab)

In [47]:
len(vocab)

267736

In [48]:
len(lm.vocab)

267736

In [54]:
vocab.lookup(train.__next__())

('Chronicles',)

In [58]:
lm.fit([train])

In [57]:
import nltk
print(nltk.__version__)

3.5


In [65]:
hypos = []
with open('../analysis.jsonl') as fin:
    for line in fin:
        hypos.append(json.loads(line.strip()))

In [98]:
from scipy.special import logsumexp

scores = 0
cnt = 0
lambda_ = 0.75
ndict = 267744
for hypo in hypos:
    knn_scores = np.array(hypo['knn_scores'])
    lm_scores = np.array(hypo['lm_scores'])
    combine = logsumexp(np.stack((knn_scores + np.log(1-lambda_), lm_scores+np.log(lambda_)), axis=-1), axis=-1)
    scores += combine.sum()
    cnt += len(hypo['knn_scores'])
    
print(np.exp(-scores / cnt))

16.736425747686383


In [110]:
np.log(2)

0.6931471805599453

In [111]:
len(hypos)

503

In [112]:
valid_d = []
ngram_scores = []
prev = []

for i, hypo in enumerate(hypos):
    local_scores = []
    if i % 10 == 0:
        print(f'processed {i} hypos')
    for tok in hypo['string']:
        prev = prev[-(ngram-1):]
        local_scores.append(lm.logscore(tok, prev) * np.log(2))
        prev.append(tok)
        
    ngram_scores.append(local_scores)

processed 0 hypos
processed 10 hypos
processed 20 hypos
processed 30 hypos
processed 40 hypos
processed 50 hypos
processed 60 hypos
processed 70 hypos
processed 80 hypos
processed 90 hypos
processed 100 hypos
processed 110 hypos
processed 120 hypos
processed 130 hypos
processed 140 hypos
processed 150 hypos
processed 160 hypos
processed 170 hypos
processed 180 hypos
processed 190 hypos
processed 200 hypos
processed 210 hypos
processed 220 hypos
processed 230 hypos
processed 240 hypos
processed 250 hypos
processed 260 hypos
processed 270 hypos
processed 280 hypos
processed 290 hypos
processed 300 hypos
processed 310 hypos
processed 320 hypos
processed 330 hypos
processed 340 hypos
processed 350 hypos
processed 360 hypos
processed 370 hypos
processed 380 hypos
processed 390 hypos
processed 400 hypos
processed 410 hypos
processed 420 hypos
processed 430 hypos
processed 440 hypos
processed 450 hypos
processed 460 hypos
processed 470 hypos
processed 480 hypos
processed 490 hypos
processed 5

In [113]:
len(ngram_scores)

503

In [120]:
print(ngram_scores[2][:20])

[-10.281385193690674, -1.726526913352188, -0.10818972796825614, -4.116738447381035, -1.8575129807038586, -6.301257375983728, -19.155008273892328, -16.055597408833552, -15.088023864807079, -6.502567791444007, -2.1866111495064295, -1.664907684801027, -11.88072916275888, -3.5768769471175963, -8.446035348312261, -5.171614862915676, -7.578745271452401, -0.0030389433221611065, -1.6749315955574957, -10.417013476840813]


In [119]:
print(hypos[2]['lm_scores'][:20])

[-9.78125, -0.76025390625, -0.02447509765625, -0.004161834716796875, -0.189208984375, -3.69921875, -1.193359375, -0.0078887939453125, -1.0322265625, -4.99609375, -0.031524658203125, -0.493408203125, -11.1171875, -2.58984375, -3.1640625, -1.048828125, -2.126953125, -0.00103759765625, -2.3125, -5.53125]


In [121]:
print(hypos[2]['knn_scores'][:20])

[-6.30859375, -1.69140625, -0.108154296875, -0.0980224609375, -0.17529296875, -3.09375, -3.49609375, -10000.0, -0.890625, -2.916015625, -0.5634765625, -0.98876953125, -10000.0, -2.72265625, -5.40234375, -2.396484375, -2.654296875, -0.0022125244140625, -1.00390625, -4.59375]


In [130]:
scores = 0
cnt = 0
lambda_ = 0.9
ndict = 267744
for hypo, ngram_s in zip(hypos, ngram_scores):
    ngram_lm_scores = np.array(ngram_s)
    lm_scores = np.array(hypo['lm_scores'])
    combine = logsumexp(np.stack((ngram_lm_scores + np.log(1-lambda_), lm_scores+np.log(lambda_)), axis=-1), axis=-1)
    scores += combine.sum()
    cnt += len(hypo['lm_scores'])
    
print(np.exp(-scores / cnt))

18.193501617872233


In [96]:
valid = ngrams(valid_d, ngram)

In [ ]:
lm.perplexity(valid)

In [ ]:
hypos[0]

In [ ]:
scores = 0
cnt = 0
lambda_ = 0.75
ndict = 267744
for hypo in hypos:
    knn_scores = np.array(hypo['knn_scores'])
    lm_scores = np.array(hypo['lm_scores'])
    combine = logsumexp(np.stack((knn_scores + np.log(1-lambda_), lm_scores+np.log(lambda_)), axis=-1), axis=-1)
    scores += combine.sum()
    cnt += len(hypo['knn_scores'])
    
print(np.exp(-scores / cnt))